<h1>0. Packages</h1>

In [1]:
import numpy as np
import re
import skyfield.sgp4lib as spg4
import matplotlib 
import PyQt5
import generate_debris as gd

matplotlib.use('Agg')
import matplotlib.pyplot as plt
from scipy.stats import uniform
from scipy.stats import norm
import scipy.stats as stats
from enum import Enum    
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from itertools import product, combinations
from matplotlib import cm

%matplotlib qt5

debris_category = Enum('Category', 'rb sc soc')

<h1>1. Data Structure</h1>

<h3>1.1 Satellite Structure</h3>

Creating an object to represent satellite from 3le.txt <br>
May end up using implementation from another package

<b>Data sources:<b><br>
https://www.space-track.org    
https://www.celestrak.com/NORAD/documentation/spacetrk.pdf

<b>Information about 3le/2le:</b><br>
https://en.wikipedia.org/wiki/Two-line_element_set#cite_note-nasahelp-12
https://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/SSOP_Help/tle_def.html

In [2]:
class two_line_element:
    
    def __init__(self, data):
        self.parse_data(data)
        
    def parse_data(self, satelite_data):
        self.title = satelite_data[0]
        line_one = satelite_data[1].split()
        line_two = satelite_data[2].split()

        self.catalog_number = re.search('[0-9]{1,5}', line_one[0]).group()
        self.classification = re.search('[UCS]', line_one[0]).group()

        international_designator = re.search('([0-9]{2})([0-9]{3})(\w{1,3})', line_one[1])
        self.launch_year = international_designator.group(1)
        self.launch_number = international_designator.group(2)
        self.launch_piece =  international_designator.group(3)

        epoch = re.search('([0-9]{2})(\d+\.\d+)', line_one[2])
        self.epoch_year = epoch.group(1)
        self.epoch_day = epoch.group(2)

        self.ballistic_coefficient = float(line_one[3])
        self.mean_motion_double_prime = line_one[4]
        self.BSTAR = line_one[5]

        self.inclination = line_two[1]
        self.right_ascension_of_ascending_node = line_two[2]
        self.eccentricity = line_two[3]
        self.argument_of_perigee = line_two[4]

        mean_anomaly = line_two[5]

<h3>Data structure implementation using spg4 from skyfield library<h3>

In [3]:
# Opening the .txt file
with open("3le.txt") as f:
    txt = f.read()
    
sat_lines = re.findall('(.*?)\n(.*?)\n(.*?)\n', txt)

# Preview of `sat_lines`, each element should be the three lines representing a satellite
print(sat_lines[0])

# Convert each group of 3 lines into a satelite object
def line_element_to_satellite(lines):
    title = lines[0]
    line1 = lines[1]
    line2 = lines[2]
    return spg4.EarthSatellite(line1, line2, name=title)

satellites = [line_element_to_satellite(lines) for lines in sat_lines]

# Verify data structure by finding element representing the ISS
iss_index = [sat.name for sat in satellites].index("0 ISS (ZARYA)")
iss = satellites[iss_index]

('0 VANGUARD 1', '1     5U 58002B   19352.86539842  .00000184  00000-0  20948-3 0  9994', '2     5  34.2610  67.7117 1845282 255.2293  83.7559 10.84789954185624')


<h3>1.2 Celestial Bodies</h3>


de421.bsp is a Ephemerides provided by JPL Horizon which has the calculated positions of celestial bodies within a certain time interval. de421 is commonly used due to its small size and its relativley up to date information

In [4]:
from skyfield.api import load, EarthSatellite
ts = load.timescale(builtin=True)

# Loading the data from de421.bsp using skyfield 
planets = load('de421.bsp')

# Finding the data about Earth
earth = planets["Earth"]

<h3>1.3 Celestial Bodies</h3>

Plotting the satellite data in a 3D view.<br>
Accidently broke my implementation so need to start with the below base and get it to plot the ISS again.

<b>Source:</b> https://space.stackexchange.com/questions/25958/how-can-i-plot-a-satellites-orbit-in-3d-from-a-tle-using-python-and-skyfield

<h1>2. Breakup Models</h1>

<h3>2.1 NASA breakup model</h3>

<b>Implementation:</b> (Found on page 207)<br>
https://www.researchgate.net/publication/295490674_Space_debris_cloud_evolution_in_Low_Earth_Orbit

<b>Alternate implementation:</b><br>
https://gitlab.obspm.fr/apetit/nasa-breakup-model/tree/master

<b>Information:</b><br>
https://digitalcommons.calpoly.edu/cgi/viewcontent.cgi?referer=https://en.wikipedia.org/&httpsredir=1&article=1094&context=theses


<b>Now implemented in file generate_debris.py</b><br>

<h3>2.2 NASA breakup model validation</h3>

In [5]:
L_c = gd.characteristic_lengths(1000, 10, 10, True, debris_category.rb)


N_fragments_total = L_c.shape[0]
lambda_c = np.log10(L_c)
areas = gd.avg_area(L_c)

def create_log_bins(values, nbins=100):
    #return np.geomspace(values.min(), values.max(), nbins)
    bins = np.geomspace(values.min(), values.max(), nbins)
    a = bins[1]/bins[0]
    bins = np.concatenate([[bins[0]/a], bins,[bins[-1]*a]])
    return bins

# Validating L_c
h, b = np.histogram(L_c, bins=create_log_bins(L_c))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$L_{c}$')
plt.ylabel(r'$N_f$')
plt.plot((b[:-1] + b[1:])/2, h, '.-')
plt.show()

# Validating Areas
h, b = np.histogram(areas, bins=create_log_bins(areas))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$Areas$')
plt.ylabel(r'$N_f$')
plt.plot((b[:-1] + b[1:])/2, h, '.-')
plt.show()

# Validating Mass & Velocity
AM = np.array(gd.distribution_AM(lambda_c, gd.debris_category.rb))
masses = areas / 10**AM

h, b = np.histogram(masses, bins=create_log_bins(masses))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$Masses$')
plt.ylabel(r'$N_f$')
plt.gca().set_yticks([0, 2, 4, 6, 8])
plt.gca().set_xticks([1e-8, 1e-5, 1e-2, 10, 1e4])
plt.gca().set_xlim([1e-8,1e4])
plt.gca().set_ylim([0,8])
plt.grid()
plt.plot((b[:-1] + b[1:])/2, h/1e5, '.-')
plt.show()

# Validating Velocity
deltaV = np.array(gd.distribution_deltaV(AM, 10, True))


h, b = np.histogram(deltaV, bins=create_log_bins(deltaV))
plt.figure(figsize=(5,5))
plt.xscale('log')
plt.xlabel(r'$\delta V$')
plt.ylabel(r'$N_f$')
plt.gca().set_yticks([0, 2, 4, 6, 8])
plt.gca().set_xticks([1e-3, 1e-2, 1e-1, 1, 10])
plt.gca().set_xlim([1e-3,10])
plt.gca().set_ylim([0,8])
plt.grid()
plt.plot((b[:-1] + b[1:])/2, h/1e5, '.-')
plt.show()

<h3>2.3 and Onward will be alternate breakup models</h3>

<h1>3. Numerical Propagation</h1>

<h3>3.1 Modeling the Atmosphere</h3>

In [6]:
import pandas as pd

excel_file = "AtmosphericModelValues.xlsx"

tabulated_values = pd.read_excel(excel_file)
tabulated_values.head()

def atmosphere_density(altitude):
    
    bins = tabulated_values['Altitude Lower Bound (km)'].values
    base_altitude = tabulated_values['Base Altitude (km)'].values
    nominal_density = tabulated_values['Nominal Density (kg/m^3)'].values
    scale_height = tabulated_values['Scale Height (km)'].values
    i = np.digitize(h, bins) - 1
    
    return nominal_density*np.exp(-(altitude-base_altitude)/(scale_height))


ModuleNotFoundError: No module named 'pandas'

<h3>3.2 Calculating Orbital Debris Properties</h3>

In [9]:
from skyfield.api import load, EarthSatellite

ts = load.timescale(builtin=True)

t = ts.now()
geocentric = iss.at(t)
init_position = geocentric.position.km

iss_mass = 419700 # kg
projectile_mass  = 227 # kg

L_c = gd.characteristic_lengths(iss_mass, projectile_mass, 100, True, debris_category.soc)
lambda_c = np.log10(L_c)
areas = gd.avg_area(L_c)
AM = np.array(gd.distribution_AM(lambda_c, debris_category.soc))
masses = areas / 10**AM
deltaV = np.array(gd.distribution_deltaV(AM, 10, True))


<h3>3.3 Converting Cartesian to Keplerian</h3>
https://downloads.rene-schwarz.com/download/M002-Cartesian_State_Vectors_to_Keplerian_Orbit_Elements.pdf

In [11]:
from numpy.linalg import norm

grav_param_earth = 398600.4418 #km^3s^-2
init_position = geocentric.position.km

deb_positions = np.empty((len(AM), 3))
deb_positions[:, :] = init_position[None,:]

deb_velocities = gd.velocity_vectors(len(AM), geocentric.velocity.km_per_s, deltaV)

orbital_momentums = np.cross(deb_positions, deb_velocities)
unit_positions = np.divide(deb_positions, norm(deb_positions, axis=1)[:, None]) ## Double check
eccentricities = (np.cross(deb_velocities, orbital_momentums) / grav_param_earth) - unit_positions

In [12]:
ascending_node_direction = np.cross(np.array([0, 0, 1])[None, :], orbital_momentums)

# Ask about costruction of true anomaly using numpy array conditions
dot_ep = np.sum(eccentricities*deb_positions, axis=1)
np.sum(eccentricities*deb_positions, axis=1)

true_anomaly = np.arccos( dot_ep / (norm(eccentricities, axis=1) * norm(deb_positions, axis=1)))
B = np.sum(deb_positions*deb_velocities, axis=1)<0
true_anomaly[B] = 2*np.pi - true_anomaly[B]


In [13]:
inclination = np.arccos(orbital_momentums[:, 2] / norm(orbital_momentums, axis=1))

eccentricities_mag = norm(eccentricities, axis = 1)
theta_ea = np.tan(true_anomaly/2) / np.sqrt((1 + eccentricities_mag)/(1 - eccentricities_mag))
eccentric_anomaly = 2*np.arctan(theta_ea)

In [14]:
# Longitude of ascending node, Omega
long_ascending_node = np.arccos(ascending_node_direction[:,0]/norm(ascending_node_direction, axis=1))
B = ascending_node_direction[:,1]<0
long_ascending_node[B] = 2*np.pi - long_ascending_node[B]

# Argument of periapsis, omega
dot_ne = np.sum(ascending_node_direction*eccentricities, axis=1)
arg_peri = np.arccos(dot_ne / (norm(ascending_node_direction, axis=1)*norm(eccentricities, axis=1)))
B = eccentricities[:,2] < 0
arg_peri[B] = 2*np.pi - arg_peri[B]

# Mean anomaly
mean_anomaly = eccentric_anomaly - norm(eccentricities,axis=1)*np.sin(eccentric_anomaly)

# Semi major axis
a = 1 / ((2/norm(deb_positions,axis=1)) - ((norm(deb_velocities, axis=1)**2) / grav_param_earth))

In [15]:
print("---- Pos. and Vel. ---")
print("pos: ", deb_positions[0,:], "km")
print("vel: ", deb_velocities[0,:], "km/s")

print("---- Orbital Elemets ---")
print("a: ", a[0]) # RIGHT
print("e: ", eccentricities_mag[0]) #RIGHT
print("i: ", inclination[0]) #RIGHT
print("Omega: ", long_ascending_node[0]) #RIGHT
print("omega: ", arg_peri[0]) #RIGHT
print("M: ", mean_anomaly[0]) # UNABLE TO VERIFY
print("v: ", true_anomaly[0]) #RIGHT

---- Pos. and Vel. ---
pos:  [-5514.18158523  3838.03380573   979.43686706] km
vel:  [-1.86084279 -4.41786539  5.90862278] km/s
---- Orbital Elemets ---
a:  6696.206566173889
e:  0.022415178417890702
i:  0.9148307760498824
Omega:  2.4210949326054156
omega:  2.441214375273645
M:  -2.2231312989157335
v:  4.02503207531102


<h3>3.4 Integrating Keplerian elements</h3>

In [16]:
# Code from https://github.com/California-Planet-Search/radvel/blob/master/radvel/kepler.py

def d_mean_anomaly(ma,a, dt_1_hr):
    return (ma + dt_1_hr*np.sqrt(grav_param_earth/a**3))

def kepler(Marr, eccarr):
    """Solve Kepler's Equation
    Args:
        Marr (array): input Mean anomaly
        eccarr (array): eccentricity
    Returns:
        array: eccentric anomaly
    """

    conv = 1.0e-12  # convergence criterion
    k = 0.85

    Earr = Marr + np.sign(np.sin(Marr)) * k * eccarr  # first guess at E
    # fiarr should go to zero when converges
    fiarr = ( Earr - eccarr * np.sin(Earr) - Marr)
    convd = np.where(np.abs(fiarr) > conv)[0]  # which indices have not converged
    nd = len(convd)  # number of unconverged elements
    count = 0

    while nd > 0:  # while unconverged elements exist
        count += 1

        M = Marr[convd]  # just the unconverged elements ...
        ecc = eccarr[convd]
        E = Earr[convd]

        fi = fiarr[convd]  # fi = E - e*np.sin(E)-M    ; should go to 0
        fip = 1 - ecc * np.cos(E)  # d/dE(fi) ;i.e.,  fi^(prime)
        fipp = ecc * np.sin(E)  # d/dE(d/dE(fi)) ;i.e.,  fi^(\prime\prime)
        fippp = 1 - fip  # d/dE(d/dE(d/dE(fi))) ;i.e.,  fi^(\prime\prime\prime)

        # first, second, and third order corrections to E
        d1 = -fi / fip
        d2 = -fi / (fip + d1 * fipp / 2.0)
        d3 = -fi / (fip + d2 * fipp / 2.0 + d2 * d2 * fippp / 6.0)
        E = E + d3
        Earr[convd] = E
        fiarr = ( Earr - eccarr * np.sin( Earr ) - Marr) # how well did we do?
        convd = np.abs(fiarr) > conv  # test for convergence
        nd = np.sum(convd is True)

    if Earr.size > 1:
        return Earr
    else:
        return Earr[0]

In [17]:
mean_anomalies = np.array([mean_anomaly])
eccentric_anomalies = np.array([eccentric_anomaly])

dt_1_hr = 60*60*1
tot_num_hours = 10

mean_anomalies = mean_anomalies.tolist()
eccentric_anomalies = eccentric_anomalies.tolist()

def orbit_integrator(num_hours):
    
    for i in range(num_hours):
        mean_anomaly_0 = mean_anomalies[-1]
        eccentric_anomaly_0 = eccentric_anomalies[-1]
    
        mean_anomaly_new = (mean_anomaly_0 + d_mean_anomaly(mean_anomaly_0, a, dt_1_hr)) % (2*np.pi)
        eccentric_anomaly_new = kepler(mean_anomaly_new, eccentricities_mag)
    
        mean_anomalies.append(mean_anomaly_new)
        eccentric_anomalies.append(eccentric_anomaly_new)
    
orbit_integrator(tot_num_hours)


In [18]:
# Convert back to arrays
mean_anomalies = np.array(mean_anomalies)
eccentric_anomalies = np.array(eccentric_anomalies)

<h3>3.5 Converting Keplerian to Cartesian</h3>

https://downloads.rene-schwarz.com/download/M001-Keplerian_Orbit_Elements_to_Cartesian_State_Vectors.pdf

In [19]:
# While unsure about using multiple times of data just will look at first

mean_anomalies = mean_anomalies[0,:]
eccentric_anomalies = eccentric_anomalies[0,:]

dist_central = a*(1 - eccentricities_mag * np.cos(eccentric_anomaly))
orbital_frame = dist_central * np.array([np.cos(true_anomaly),
                                         np.sin(true_anomaly),
                                         np.zeros(len(AM))])
orbital_prime_frame = (np.sqrt(grav_param_earth * a) / dist_central) * np.array([-np.sin(eccentric_anomaly),
                                                                                 (np.sqrt(1 - eccentricities_mag**2))*np.cos(eccentric_anomaly),
                                                                                 np.zeros(len(AM))])

In [20]:
positions = np.array([
    orbital_frame[0] * (np.cos(arg_peri)*np.cos(long_ascending_node) - np.sin(arg_peri)*np.cos(inclination)*np.sin(long_ascending_node)) - orbital_frame[1] * (np.sin(arg_peri)*np.cos(long_ascending_node) + np.cos(arg_peri)*np.cos(inclination)*np.sin(long_ascending_node)),
    orbital_frame[0] * (np.cos(arg_peri)*np.sin(long_ascending_node) + np.sin(arg_peri)*np.cos(inclination)*np.cos(long_ascending_node)) + orbital_frame[1] * (np.cos(arg_peri)*np.cos(inclination)*np.cos(long_ascending_node) - np.sin(arg_peri)*np.sin(long_ascending_node)),
    orbital_frame[0] * (np.sin(arg_peri)*np.sin(inclination)) + orbital_frame[1] * (np.cos(arg_peri)*np.sin(inclination))
])

positions = positions.T

velocities = np.array([
    orbital_prime_frame[0] * (np.cos(arg_peri)*np.cos(long_ascending_node) - np.sin(arg_peri)*np.cos(inclination)*np.sin(long_ascending_node)) - orbital_prime_frame[1] * (np.sin(arg_peri)*np.cos(long_ascending_node) + np.cos(arg_peri)*np.cos(inclination)*np.sin(long_ascending_node)),
    orbital_prime_frame[0] * (np.cos(arg_peri)*np.sin(long_ascending_node) + np.sin(arg_peri)*np.cos(inclination)*np.cos(long_ascending_node)) + orbital_prime_frame[1] * (np.cos(arg_peri)*np.cos(inclination)*np.cos(long_ascending_node) - np.sin(arg_peri)*np.sin(long_ascending_node)),
    orbital_prime_frame[0] * (np.sin(arg_peri)*np.sin(inclination)) + orbital_prime_frame[1] * (np.cos(arg_peri)*np.sin(inclination))
])

velocities = velocities.T

# NEED TO ASK ABOUT INTEGRATION AND ARRAY FRIENDLY CONVERSION
<h3>3.6 Performing the integration </h3>

<h1>4. Visualization </h1>

In [21]:
# Creating 3D plot showing orbital information
earth_radius = 6378.0 #km

def create_3D_plot():
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    
    # Plot central body
    _u, _v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
    _x = earth_radius*np.cos(_u)*np.sin(_v)
    _y = earth_radius*np.sin(_u)*np.sin(_v)
    _z = earth_radius*np.cos(_v)
    ax.plot_surface(_x, _y, _z, cmap=cm.hot)
    
    # Plot axis
    unit = earth_radius * 2
    x,y,z = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    u, v, w =[[unit, 0, 0], [0, unit, 0], [0, 0, unit]]
    ax.quiver(x,y,z, u, v, w, color='k')
    
    ax.set_xlim([-2.1*earth_radius, 2.1*earth_radius])
    ax.set_ylim([-2.1*earth_radius, 2.1*earth_radius])
    ax.set_zlim([-2.1*earth_radius, 2.1*earth_radius])
    
    # Plot debris
    posititions_subset = positions[:1000, :]
    ax.scatter(posititions_subset[:, 0], posititions_subset[:, 1], posititions_subset[: ,2], 'k', label="Debris")    
    plt.show()

    
create_3D_plot()

In [ ]:
# import rebound
# sim = rebound.Simulation()

In [ ]:
# sim.getWidget(scale=10000, size=(500,500))

In [ ]:
# sim.move_to_com()

# print(debris_positions[0])

In [ ]:
# earth_mass = 5.972E24 #kg
# sim.add(m=earth_mass) # add earth
# for i in range(len(deb_velocities)):
#     pos = deb_positions[i]
#     vel = deb_velocities[i]
#     sim.add(a=a[i], e=eccentricities_mag[i], omega=arg_peri[i], M=mean_anomaly[i], inc=inclination[i])
#     if i > 50000:
#         break
# for i in range(len(sim.particles)):
#     if sim.particles[i].m == 0:
#         sim.particles[i].test_particle_type = 1

In [ ]:
# print(sim.N)                # Gets the current number of particles
# print(sim.N_active)  

In [ ]:
# from IPython.display import display, clear_output
# import matplotlib.pyplot as plt
# sim.move_to_com()
# sim.dt = 1e-3
# sim.integrator = "whfast"
# for i in range(300):
#     sim.integrate(sim.t+1e-9)

In [ ]:
# x = [sim.particles[i].x for i in range(len(sim.particles))]
# y = [sim.particles[i].y for i in range(len(sim.particles))]
# fig = plt.figure(figsize=(5,5))
# ax = plt.subplot(111)
# plt.scatter(x, y, marker='.', color='k', s=1.2);

In [ ]:
# from IPython.display import display, clear_output
# import matplotlib.pyplot as plt
# sim.move_to_com()
# for i in range(100):
#     sim.integrate(sim.t+1e-9)
#     fig, ax = rebound.OrbitPlot(sim,color=True,unitlabel="[km]")
#     display(fig)
#     plt.close(fig)
#     clear_output(wait=True)

In [ ]:
# frame = OrbitPlotter3D()

# frame.plot(molniya)
# frame.plot(iss)

In [ ]:
# import poliastro

In [ ]:
# from poliastro.twobody import Orbit
# from poliastro.bodies import Earth, Sun
# from astropy import units as u


# # This function cannot handle vectors, inherriently slow, will need to write own array friedly implementation
# orbits = [ Orbit.from_classical(Earth, a[i]*u.kilometer, eccentricities_mag[i]*u.one, inclination[i]*u.rad, long_ascending_node[i]*u.rad, arg_peri[i]*u.rad, true_anomaly[i]*u.rad) for i in range(10000)]
    
    

In [ ]:
# from poliastro.plotting import *

# frame = OrbitPlotter3D()

# for i in range(100):
#     frame.plot(orbits[i])

In [ ]:
# frame.plot(orbits[101])